# pybela tutorial
In this workshop we'll be using jupyter notebooks and python to:
1. Record a dataset of potentiometer sensor values
2. Train a TCN to predict those values
3. Cross-compile and deploy the model to run in real-time in Bela

Connect your Bela to the laptop and run the cell below:

In [ ]:
! ssh-keyscan $BBB_HOSTNAME >> ~/.ssh/known_hosts

Let's also import all the necessary python libraries:

In [ ]:
import os
from pybela import Logger
import asyncio

import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm 

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

## 1 – pybela basics

[pybela](https://github.com/BelaPlatform/pybela/) allows sending data back and forth between python and Bela.

For pybela to be able to communicate with Bela, there has to be a project running on the Bela. 

We have an example project in `/root/bela-code/pybela-basic`. Let's take a look at the cpp code.

### c++ code
The cpp code in `pybela-basics/render.cpp` reads the value of a potentiometer which controls the volume of a wave sound. The potentiometer value is stored in a `pot` variable which is defined in a special way so we can access it from python.

You should connect a potentiometer to the Bela's analog input 0:

<p align="center">
  <img src="_fritzing/potentiometer.png" alt="potentiometer" width="400"/>
</p>
Let's take a look at the Watcher API in `pybela-basics/render.cpp`:


The Watcher API in the Bela code allows "watching" variables in the Bela code so we can retrieve their values from python. First, we define the variables we want to watch this way:

```cpp
Watcher<float> pot("pot"); // the "pot" variable is "watched"
```

In the `setup()` function, we initialize the Watcher:

```cpp
bool setup(BelaContext *context, void *userData) {

  Bela_getDefaultWatcherManager()->getGui().setup(context->projectName);
  Bela_getDefaultWatcherManager()->setup(
      context->audioSampleRate); // set sample rate in watcher

      ...
```

We need to tell the Watcher the rate at which we want to observe the variables. For that, we "tick" the Watcher clock in the `render()` function. Note that we only tick it at the analog rate, since "pot" is an analog variable (typicially read once per two audio frames):

```cpp

void render(BelaContext *context, void *userData) {

  for (unsigned int n = 0; n < context->audioFrames; n++) {

    uint64_t analogFramesElapsed = int((context->audioFramesElapsed + n) / 2);
    Bela_getDefaultWatcherManager()->tick(
        analogFramesElapsed); // tick the watcher clock

    if (gAudioFramesPerAnalogFrame && !(n % gAudioFramesPerAnalogFrame)) {
      pot = analogRead(context, n / gAudioFramesPerAnalogFrame, 0);
    }
  }
}
```

Let's now crosscompile this code and run it on the Bela.

### xcompiling the cpp code 

To cross-compile the code, we use `cmake` and a cross-compilation toolchain. A cross-compilation toolchain tells `cmake` that even though we are compiling the code on our laptop, the code is meant to run on the Bela.


In [ ]:
!cd bela-code/pybela-basic && cmake -S . -B build -DPROJECT_NAME=pybela-basic -DCMAKE_TOOLCHAIN_FILE=/sysroot/root/Bela/Toolchain.cmake
!cd bela-code/pybela-basic && cmake --build build -j

We have now built an executable for the Bela, which is located at `bela-code/pybela-basic/build/pybela-basic`. Let's copy it to the Bela, along with the project files so we can access them from the Bela IDE, and the `waves.wav` file which is used by the project.

In [ ]:
!rsync -rvL --timeout 10 bela-code/pybela-basic/build/pybela-basic root@$BBB_HOSTNAME:Bela/projects/pybela-basic/
!rsync -rvL --timeout 10 bela-code/pybela-basic/  --exclude="build" root@$BBB_HOSTNAME:/root/Bela/projects/pybela-basic/

To run it, open a terminal and ssh into the Bela and run the program:

```bash
ssh root@bela.local
cd Bela/projects/pybela-basic && ./pybela-basic
```
(running this on the Jupyter notebook would block the cell and we need to be able to run the next cells!)

### python code
Now we are ready to interact with the Bela code from python. First we import `pybela` and create a `Logger` object:

In [ ]:
logger=Logger(ip=os.environ["BBB_HOSTNAME"])
logger.connect()

Now the Logger is connected to Bela. The Logger class allows us recording datasets locally in Bela and transferring them automatically to the host computer. 

Connect your headphones to the Bela audio output and run the cell below while you rotate the potentiometer.

In [ ]:
file_paths = logger.start_logging("pot")

After a few seconds, you can stop the logging:

In [ ]:
logger.stop_logging()

Once the transfer is done, you can retrieve the logged data by reading the binary file in which it was saved. That binary file stores the data as timestamped buffers, which we are not interested on as we just want a continuous array of potentiometer values.

In [ ]:
raw = logger.read_binary_file(
        file_path=file_paths["local_paths"]["pot"], timestamp_mode=logger.get_prop_of_var("pot", "timestamp_mode"))
data = [data for _buffer in raw["buffers"] for data in _buffer["data"]]

We can now plot the data using matplotlib.

In [ ]:
analog_sample_rate = logger.sample_rate/2

plt.plot(np.arange(len(data)) / analog_sample_rate, data)
plt.title('Pot Data')
plt.xlabel('Time')
plt.ylabel('Amplitude')

## 2 – potentiometers dataset capture

We are now ready to record a dataset with two potentiometers. Connect the second potentiometer to your Bela:

<p align="center">
  <img src="_fritzing/potentiometer_2.png" alt="potentiometer" width="300"/>
</p>

 We will be running the `dataset-capture` project. Now the first potentiometer controls the waveshape of an LFO and the second potentiometer, the volume of the sound.

Let's start by cross-compiling the code and copying it to Bela.

In [ ]:
!cd bela-code/dataset-capture && cmake -S . -B build -DPROJECT_NAME=dataset-capture -DCMAKE_TOOLCHAIN_FILE=/sysroot/root/Bela/Toolchain.cmake
!cd bela-code/dataset-capture && cmake --build build -j

In [ ]:
!rsync -rvL --timeout 10 bela-code/dataset-capture/build/dataset-capture root@$BBB_HOSTNAME:Bela/projects/dataset-capture/
!rsync -rvL --timeout 10 bela-code/dataset-capture/  --exclude="build" root@$BBB_HOSTNAME:/root/Bela/projects/dataset-capture/

Now you can run the `dataset-capture` project on the Bela:

```bash
ssh root@bela.local
cd Bela/projects/dataset-capture && ./dataset-capture
```

Feel free to play around with the potentiometer and the piezo sensor. You can also edit the code in the IDE and re-run the project.

Once you're ready. You can record a dataset of potentiometer and piezo sensor values.

In [ ]:
logger=Logger(ip=os.environ["BBB_HOSTNAME"])
logger.connect()

You can time the length of your dataset using `asyncio.sleep(time_in_seconds)`. Note we are not using `time.sleep()` because it would block the Jupyter notebook.

In [ ]:
file_paths = logger.start_logging(variables=["pot1", "pot2"])
await asyncio.sleep(90)
logger.stop_logging()

In [ ]:
pot1_raw_data = logger.read_binary_file(
        file_path=file_paths["local_paths"]["pot1"], timestamp_mode=logger.get_prop_of_var("pot1", "timestamp_mode"))
pot2_raw_data = logger.read_binary_file(
        file_path=file_paths["local_paths"]["pot2"], timestamp_mode=logger.get_prop_of_var("pot2", "timestamp_mode"))

pot1_data = [data for _buffer in pot1_raw_data["buffers"] for data in _buffer["data"]]
pot2_data = [data for _buffer in pot2_raw_data["buffers"] for data in _buffer["data"]]

We can now plot the data using matplotlib.

In [ ]:
analog_sample_rate = logger.sample_rate/2

plt.figure(figsize=(10, 8))

plt.subplot(2, 1, 1)
plt.plot(np.arange(len(pot1_data)) / analog_sample_rate, pot1_data)
plt.title('Pot 1 Data')
plt.xlabel('Time')
plt.ylabel('Amplitude')

# Second subplot for pie_data
plt.subplot(2, 1, 2)
plt.plot(np.arange(len(pot2_data)) / analog_sample_rate, pot2_data)
plt.title('Pot 2 Data')
plt.xlabel('Time')
plt.ylabel('Amplitude')
 
plt.tight_layout()
plt.show()

## 3 - train model
Now we are ready to train our model.
We can generate a pytorch compatible dataset using the `SensorDataset` class. This class divides the data you recorded previously in sequences of 512 values.

In [ ]:
seq_len = 512
batch_size = 32
target_windows = 1

class SensorDataset(Dataset):
    def __init__(self, pot1_data, pot2_data, seq_len, target_windows):
        super().__init__()
        
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        # make len divisible by seq_len
        _len = min(len(pot1_data), len(pot2_data))
        _len = _len - (_len % seq_len)
        pot1_data, pot2_data = pot1_data[:_len], pot2_data[:_len]
        
        pot1_sequences = torch.FloatTensor([pot1_data[i:i+seq_len] for i in range(0, len(pot1_data), seq_len)])
        pot2_sequences = torch.FloatTensor([pot2_data[i:i+seq_len] for i in range(0, len(pot2_data), seq_len)])

        self.inputs = torch.stack((pot1_sequences[:-target_windows], pot2_sequences[:-target_windows]), dim=2).to(self.device)
        outputs=[]
        for idx in range(1, len(pot1_sequences)-target_windows+1):
            tgt_seq = torch.stack((pot1_sequences[idx:target_windows+idx].flatten(), pot2_sequences[idx:target_windows+idx].flatten()), dim=1)
            outputs.append(tgt_seq)
        self.outputs = torch.stack(outputs).to(self.device)
        
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, i):
        return self.inputs[i], self.outputs[i]
    
dataset = SensorDataset(pot1_data, pot2_data, seq_len, target_windows)
dataset_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

Below we define a TCN. We will use an Adam optimiser with a learning rate of 0.001 and use the mean square error as loss.

In [ ]:
class Chomp1d(nn.Module):
    """Layer that removes trailing values to ensure causality in the TCN."""
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()

class TemporalBlock(nn.Module):
    """A single temporal block in a TCN, with dilated causal convolutions and residual connections."""
    def __init__(self, in_channels, out_channels, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, stride=stride, padding=padding, dilation=dilation)
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size, stride=stride, padding=padding, dilation=dilation)
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
                                 self.conv2, self.chomp2, self.relu2, self.dropout2)
        self.downsample = nn.Conv1d(in_channels, out_channels, 1) if in_channels != out_channels else None
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)

class TemporalConvNet(nn.Module):
    """A Temporal Convolutional Network (TCN) made up of multiple temporal blocks."""
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2, upsample_factor=3):
        super(TemporalConvNet, self).__init__()
        self.upsample_factor = upsample_factor  # Upsample factor as a parameter
        
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers.append(TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                        padding=(kernel_size-1) * dilation_size, dropout=dropout))

        self.network = nn.Sequential(*layers)
        
        # Upsample layer to increase sequence length by the upsample factor
        self.upsample_layer = nn.ConvTranspose1d(num_channels[-1], num_channels[-1], kernel_size=self.upsample_factor, stride=self.upsample_factor)
        
        # Output layer to map back to the input feature size
        self.output_layer = nn.Conv1d(num_channels[-1], num_inputs, 1)  

    def forward(self, x):
        # Input shape: [batch_size, sequence_len, feature_size]
        x = x.transpose(1, 2)  # Change shape to [batch_size, feature_size, sequence_len]
        y = self.network(x)
        
        # Upsample the sequence length by the upsample factor
        y = self.upsample_layer(y)
        
        # Map back to the original feature size
        y = self.output_layer(y)  
        y = y.transpose(1, 2)  # Change shape back to [batch_size, sequence_len*upsample_factor, feature_size]
        return y


batch_size, sequence_len, feature_size = 32, 512, 2
upsample_factor = target_windows  # Define the upsample factor
model = TemporalConvNet(num_inputs=feature_size, num_channels=[16, 32, 16], kernel_size=3, dropout=0.2, upsample_factor=upsample_factor)

# Create a random tensor of shape [batch_size, sequence_len, feature_size]
x = torch.randn(batch_size, sequence_len, feature_size)

# Forward pass through the model
output = model(x)

print(output.shape)  # Output shape should be [batch_size, sequence_len*upsample_factor, feature_size]


We can now train our model:

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.MSELoss(reduction='mean')

epochs = 50

print("Running on device: {}".format(device))

epoch_losses = np.array([])
for epoch in range(1, epochs+1):

    print(">> Epoch: {} <<".format(epoch))

    # training loop
    batch_losses = np.array([])
    model.train()

    for batch_idx, (data, targets) in enumerate(tqdm(dataset_loader)):
        # (batch_size, seq_len, input_size)
        data = data.to(device=device, non_blocking=True)
        # (batch_size, seq_len, input_size)
        targets = targets.to(device=device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)  # lower memory footprint
        out = model(data)
        loss = torch.sqrt(criterion(out, targets))
        batch_losses = np.append(batch_losses, loss.item())
        loss.backward()
        optimizer.step()
    
    epoch_losses = np.append(epoch_losses, batch_losses.mean().round(4))

    print(f'Loss: {epoch_losses[-1]}')

We can plot the loss to see how the training went:

In [ ]:
x_epochs = range(1, epochs + 1)

plt.scatter(x_epochs, epoch_losses, marker='o')
plt.plot(x_epochs, epoch_losses, linestyle='-')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.xticks(x_epochs)  # Ensure x-axis has integer values for each epoch
plt.title('Training Loss per Epoch')
plt.show()

Let's make sure the model trained correctly by visualising some of the predictions.

In [ ]:
# Select random indexes for plotting
num_examples = 4
random_indexes = np.random.choice(len(dataset), size=num_examples, replace=False)

# Calculate the number of rows for the subplots
num_rows = num_examples

# Set up subplots
fig, axes = plt.subplots(num_rows, 2, figsize=(12, 3 * num_rows))

# Loop through random indexes and plot predictions
for idx, ax_row in zip(random_indexes, axes):
    input, target = dataset.__getitem__(idx)
    output = model(input.unsqueeze(0))
    
    # Plot for the first dimension in the first column
    ax_row[0].plot(target[:, 0].detach().cpu(), label='Target')
    ax_row[0].plot(output[0, :, 0].detach().cpu(), label='Predictions')
    ax_row[0].set_xlabel('Time')
    ax_row[0].set_ylabel('Value')
    ax_row[0].legend()
    ax_row[0].set_ylim(0, 3)
    ax_row[0].set_title(f'Figure for Index {idx} - Pot 1')
    
    # Plot for the second dimension in the second column
    ax_row[1].plot(target[:, 1].detach().cpu(), label='Target')
    ax_row[1].plot(output[0, :, 1].detach().cpu(), label='Prediction')
    ax_row[1].set_xlabel('Time')
    ax_row[1].set_ylabel('Value')
    ax_row[1].legend()
    ax_row[1].set_title(f'Figure for Index {idx} - Pot 2')

plt.tight_layout()
plt.show()

When you're ready, save the model so that we can export it into Bela.

In [ ]:
model.to(device='cpu')
model.eval()
script = torch.jit.script(model)
path = "bela-code/inference/model.jit"
script.save(path)

In [ ]:
torch.jit.load(path) # check model is properly saved

## 4 - deploy and run

The cell below will cross-compile and deploy the project to Bela.

In [ ]:
!cd bela-code/inference && cmake -S . -B build -DPROJECT_NAME=inference -DCMAKE_TOOLCHAIN_FILE=/sysroot/root/Bela/Toolchain.cmake
!cd bela-code/inference && cmake --build build -j

In [ ]:
!rsync -rvL --timeout 10 bela-code/inference/build/inference root@$BBB_HOSTNAME:Bela/projects/inference/
!rsync -rvL --timeout 10 bela-code/inference/  --exclude="build" root@$BBB_HOSTNAME:/root/Bela/projects/inference/

Once deployed, you can run it from the Bela terminal (which you can access from your regular terminal typing `ssh root@bela.local`) by typing:
```bash
cd Bela/projects/inference
./pot-inference -m model.jit
```